# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 2 2022 11:15AM,249882,19,8724054,"Graystone, LLC",Released
1,Nov 2 2022 11:15AM,249882,19,8724057,"Graystone, LLC",Released
2,Nov 2 2022 11:15AM,249882,19,8724062,"Graystone, LLC",Released
3,Nov 2 2022 11:15AM,249882,19,8724063,"Graystone, LLC",Released
4,Nov 2 2022 11:10AM,249881,21,631710,"NBTY Global, Inc.",Released
5,Nov 2 2022 11:09AM,249880,21,631709,"NBTY Global, Inc.",Released
6,Nov 2 2022 11:05AM,249879,19,8724077,"Graystone, LLC",Released
7,Nov 2 2022 10:58AM,249877,10,MSP215425,"Methapharm, Inc.",Released
8,Nov 2 2022 10:58AM,249877,10,MSP215428,"Methapharm, Inc.",Released
9,Nov 2 2022 10:58AM,249877,10,MSP215429,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,249877,Released,13
37,249879,Released,1
38,249880,Released,1
39,249881,Released,1
40,249882,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249877,NaN,NaN,13.0
249879,NaN,NaN,1.0
249880,NaN,NaN,1.0
249881,NaN,NaN,1.0
249882,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249828,0.0,0.0,1.0
249831,0.0,1.0,0.0
249837,0.0,0.0,2.0
249839,0.0,4.0,1.0
249846,0.0,2.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249828,0,0,1
249831,0,1,0
249837,0,0,2
249839,0,4,1
249846,0,2,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249828,0,0,1
1,249831,0,1,0
2,249837,0,0,2
3,249839,0,4,1
4,249846,0,2,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249828,,,1
1,249831,,1,
2,249837,,,2
3,249839,,4,1
4,249846,,2,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 2 2022 11:15AM,249882,19,"Graystone, LLC"
4,Nov 2 2022 11:10AM,249881,21,"NBTY Global, Inc."
5,Nov 2 2022 11:09AM,249880,21,"NBTY Global, Inc."
6,Nov 2 2022 11:05AM,249879,19,"Graystone, LLC"
7,Nov 2 2022 10:58AM,249877,10,"Methapharm, Inc."
20,Nov 2 2022 10:58AM,249869,15,"Person & Covey, Inc."
23,Nov 2 2022 10:56AM,249876,10,"Methapharm, Inc."
27,Nov 2 2022 10:53AM,249874,16,Sartorius Bioprocess Solutions
28,Nov 2 2022 10:48AM,249872,10,ISDIN Corporation
57,Nov 2 2022 10:43AM,249871,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 2 2022 11:15AM,249882,19,"Graystone, LLC",,,4
1,Nov 2 2022 11:10AM,249881,21,"NBTY Global, Inc.",,,1
2,Nov 2 2022 11:09AM,249880,21,"NBTY Global, Inc.",,,1
3,Nov 2 2022 11:05AM,249879,19,"Graystone, LLC",,,1
4,Nov 2 2022 10:58AM,249877,10,"Methapharm, Inc.",,,13
5,Nov 2 2022 10:58AM,249869,15,"Person & Covey, Inc.",,,3
6,Nov 2 2022 10:56AM,249876,10,"Methapharm, Inc.",,,4
7,Nov 2 2022 10:53AM,249874,16,Sartorius Bioprocess Solutions,,,1
8,Nov 2 2022 10:48AM,249872,10,ISDIN Corporation,,,29
9,Nov 2 2022 10:43AM,249871,10,ISDIN Corporation,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 11:15AM,249882,19,"Graystone, LLC",4,,
1,Nov 2 2022 11:10AM,249881,21,"NBTY Global, Inc.",1,,
2,Nov 2 2022 11:09AM,249880,21,"NBTY Global, Inc.",1,,
3,Nov 2 2022 11:05AM,249879,19,"Graystone, LLC",1,,
4,Nov 2 2022 10:58AM,249877,10,"Methapharm, Inc.",13,,
5,Nov 2 2022 10:58AM,249869,15,"Person & Covey, Inc.",3,,
6,Nov 2 2022 10:56AM,249876,10,"Methapharm, Inc.",4,,
7,Nov 2 2022 10:53AM,249874,16,Sartorius Bioprocess Solutions,1,,
8,Nov 2 2022 10:48AM,249872,10,ISDIN Corporation,29,,
9,Nov 2 2022 10:43AM,249871,10,ISDIN Corporation,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 11:15AM,249882,19,"Graystone, LLC",4,,
1,Nov 2 2022 11:10AM,249881,21,"NBTY Global, Inc.",1,,
2,Nov 2 2022 11:09AM,249880,21,"NBTY Global, Inc.",1,,
3,Nov 2 2022 11:05AM,249879,19,"Graystone, LLC",1,,
4,Nov 2 2022 10:58AM,249877,10,"Methapharm, Inc.",13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 11:15AM,249882,19,"Graystone, LLC",4.0,NaN,NaN
1,Nov 2 2022 11:10AM,249881,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Nov 2 2022 11:09AM,249880,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Nov 2 2022 11:05AM,249879,19,"Graystone, LLC",1.0,NaN,NaN
4,Nov 2 2022 10:58AM,249877,10,"Methapharm, Inc.",13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 11:15AM,249882,19,"Graystone, LLC",4.0,0.0,0.0
1,Nov 2 2022 11:10AM,249881,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Nov 2 2022 11:09AM,249880,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Nov 2 2022 11:05AM,249879,19,"Graystone, LLC",1.0,0.0,0.0
4,Nov 2 2022 10:58AM,249877,10,"Methapharm, Inc.",13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3248193,137.0,3.0,4.0
15,1998871,52.0,42.0,0.0
16,499737,1.0,1.0,0.0
19,1499132,8.0,7.0,0.0
21,499761,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3248193,137.0,3.0,4.0
1,15,1998871,52.0,42.0,0.0
2,16,499737,1.0,1.0,0.0
3,19,1499132,8.0,7.0,0.0
4,21,499761,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,137.0,3.0,4.0
1,15,52.0,42.0,0.0
2,16,1.0,1.0,0.0
3,19,8.0,7.0,0.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,137.0
1,15,Released,52.0
2,16,Released,1.0
3,19,Released,8.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,4.0,0.0,0.0,0.0,0.0
Executing,3.0,42.0,1.0,7.0,0.0
Released,137.0,52.0,1.0,8.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,4.0,0.0,0.0,0.0,0.0
1,Executing,3.0,42.0,1.0,7.0,0.0
2,Released,137.0,52.0,1.0,8.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,4.0,0.0,0.0,0.0,0.0
1,Executing,3.0,42.0,1.0,7.0,0.0
2,Released,137.0,52.0,1.0,8.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()